# practice

### torch

In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
import importlib
from config.config_manager import _C as cfg
from tools.utils import deploy_macro
cfg.merge_from_file("../keypoint.yml")
cfg.RESUME = ""
cfg.MODEL.NORM = 'ABN'
deploy_macro(cfg)

In [2]:
from data.build_loader import build_coco_person_loader
from model.managers.manager_center import CenterManager

In [3]:
train_loader, val_loader = build_coco_person_loader(cfg)
manager = CenterManager(cfg)

loading annotations into memory...
Done (t=5.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


In [4]:
manager.use_multigpu()
model = manager.model

In [5]:
model.backbone.hg_mods[0].low2

hg_module(
  (up1): Sequential(
    (0): fire_module(
      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=identity)
      (conv_1x1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv_3x3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (bn2): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=identity)
      (relu): LeakyReLU(negative_slope=0.01, inplace)
    )
    (1): fire_module(
      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=identity)
      (conv_1x1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv_3x3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (bn2): InPlaceABN(256, eps=1e-05, momentum=0.1, affi

In [6]:
batch = next(iter(train_loader))

In [7]:
for key in batch.keys():
    batch[key] = batch[key].cuda()

In [12]:
images = batch['inp'].cuda()
feats = manager.model(images) 


In [9]:
pre_out = model.backbone.pre(images)
hg1_up1_out = model.backbone.hg_mods[0].up1(pre_out)
hg1_low1_out = model.backbone.hg_mods[0].low1(hg1_up1_out)

In [10]:
hg1_low1_out.shape

torch.Size([1, 256, 32, 32])

In [11]:
hg1_low1_out.sum().sigmoid().backward()

In [8]:
batch.keys()

dict_keys(['inp', 'hm', 'wh', 'reg', 'reg_mask', 'ind'])

In [13]:
a, b = manager.loss_func(feats, batch)
# self.total_loss.backward()

In [29]:
feats[0]['reg'].shape

torch.Size([1, 2, 64, 64])

In [14]:
a.backward()

In [15]:
num_params = 0
for layer, p in model.named_parameters():

    if not p.requires_grad:
        continue
    
    num_params += p.numel()

In [16]:
print("Trainable parameters: {:.2f}M".format(num_params / 1000000.0))

Trainable parameters: 25.13M


In [17]:
a = torch.rand(1,3,512,512)
model = manager.model
model.train()

Model(
  (backbone): HourglassNet(
    (pre): Sequential(
      (0): convolution(
        (conv): Conv2d(3, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      )
      (1): residual(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        (skip): Sequential(
          (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): InPlaceABN(256, eps=1e-05, momentum=0.1, affine=True, activation=identity)
        )
        (relu): LeakyReLU(negative_slope=0.01, inplace)
      )
      (2): residual(
        (conv1): Conv2d(

In [18]:
model.heads

AttributeError: 'Model' object has no attribute 'heads'

In [19]:
out = model(a.cuda())

[{'hm': tensor([[[[-2.2494, -2.2093, -2.2957,  ..., -2.1749, -2.2129, -2.1252],
            [-2.1748, -2.1643, -2.2896,  ..., -2.1925, -2.2053, -2.1763],
            [-2.2744, -2.3161, -2.1912,  ..., -2.2263, -2.1701, -2.1511],
            ...,
            [-2.0581, -2.0498, -2.1303,  ..., -2.2343, -2.1853, -2.1726],
            [-2.1471, -2.0332, -2.0911,  ..., -2.1924, -2.2166, -2.1311],
            [-2.1272, -2.1599, -2.0723,  ..., -2.1747, -2.1571, -2.1465]]]],
         device='cuda:0', grad_fn=<CudnnConvolutionBackward>),
  'wh': tensor([[[[-0.1594, -0.0312, -0.2108,  ...,  0.0093, -0.0513, -0.0817],
            [-0.1555, -0.0910, -0.1681,  ..., -0.0044,  0.0057, -0.0378],
            [-0.1741, -0.1662, -0.1543,  ..., -0.0533,  0.0912, -0.0051],
            ...,
            [ 0.0124,  0.0208,  0.0105,  ..., -0.0302,  0.0594,  0.0097],
            [ 0.0253,  0.0664,  0.0331,  ..., -0.0059,  0.0351,  0.0471],
            [ 0.0057,  0.0295, -0.0183,  ..., -0.0148,  0.0399, -0.0286]],

In [12]:
out[-1]

{'hm': tensor([[[[  115.4680,  -392.3594,   334.7852,  ...,  -376.1155,
               35.9382,  -213.9702],
           [ -248.1862,  -124.4669,  -217.4783,  ...,   216.5553,
             -145.1919,   231.8074],
           [   15.1993,  -173.6681,   -26.8982,  ..., -1134.5842,
              167.5008,  -186.2161],
           ...,
           [  196.9365,  -217.4518,   243.7393,  ...,  -400.8627,
               62.2878,   515.2735],
           [  652.4150,  -492.6696,  -289.9304,  ..., -2323.4958,
             -366.9826,  -329.8666],
           [   33.6572,   138.9205,  -601.0677,  ...,    71.5848,
             -119.8921,   451.1734]]]], device='cuda:0'),
 'wh': tensor([[[[  172.3305,  -178.0389,   257.0131,  ...,  -406.8870,
             -123.4730,  -160.6042],
           [ -111.5011,   -12.0172,    60.4420,  ...,  -390.6871,
              150.5281,  -313.6488],
           [  333.1461,   514.1946,   277.1033,  ...,   184.2935,
             -340.0359,   -61.7338],
           ...,
        

In [7]:
cache_file = '/home/allen/CornerNet_Squeeze_500000.pkl'
print("loading model from {}".format(cache_file))
with open(cache_file, "rb") as f:
    state = torch.load(f)
    

loading model from /home/allen/CornerNet_Squeeze_500000.pkl


In [8]:
src_state = state
dst_state = model.state_dict()

In [9]:
dst_layers = []
for key in dst_state.keys():
    if 'pre' in key or 'hg_mods' in key or 'cnvs' in key or 'inters' in key:
        dst_layers.append(key)

In [10]:
src_layers = []
for key in src_state.keys():
    if 'pre' in key or 'hgs' in key or 'cnvs' in key or 'inters' in key:
        if 'num_batches_tracked' not in key:
            src_layers.append(key)

In [13]:
for dst, src in zip(dst_layers, src_layers):
    if dst_state[dst].size() == src_state[src].size():
        dst_state[dst] = src_state[src]
    else:
        print(dst, src)

In [15]:
src_state[src]

tensor([1.4887, 0.4198, 0.5426, 0.5092, 0.4947, 0.3634, 0.3661, 0.2819, 0.3378,
        0.5425, 0.5796, 0.5219, 0.3570, 0.2839, 0.3557, 0.4213, 0.6932, 0.3933,
        0.6667, 0.5858, 0.5530, 0.4547, 0.4733, 0.3944, 0.3859, 0.4886, 0.4198,
        0.6582, 0.4538, 0.3808, 0.6788, 0.6328, 0.8631, 0.4983, 0.5490, 0.3519,
        0.5631, 0.4594, 0.5365, 0.5248, 0.4451, 0.4558, 0.4793, 0.6389, 0.7234,
        0.3734, 0.3457, 0.5360, 0.4281, 1.1684, 0.5294, 0.4066, 0.3297, 0.3671,
        0.6113, 0.4425, 0.4684, 0.6815, 0.3859, 0.6053, 0.3822, 0.3913, 1.2453,
        0.4409, 0.3566, 0.4454, 0.4176, 0.4760, 0.4971, 0.5012, 0.4316, 0.3986,
        0.3751, 0.3391, 0.3834, 0.4656, 0.4162, 0.7432, 0.5051, 0.6335, 0.4490,
        0.3647, 0.4829, 0.8224, 0.3247, 0.3659, 0.4589, 0.3837, 0.4440, 0.4366,
        0.5135, 0.5253, 0.4228, 0.3761, 0.4436, 0.4647, 0.5121, 0.4203, 0.4344,
        0.3311, 0.6444, 0.4212, 0.5446, 0.2708, 0.4052, 0.3880, 0.3720, 0.7506,
        0.4735, 0.4232, 0.3548, 0.5058, 

In [14]:
dst_state[dst]

tensor([1.4887, 0.4198, 0.5426, 0.5092, 0.4947, 0.3634, 0.3661, 0.2819, 0.3378,
        0.5425, 0.5796, 0.5219, 0.3570, 0.2839, 0.3557, 0.4213, 0.6932, 0.3933,
        0.6667, 0.5858, 0.5530, 0.4547, 0.4733, 0.3944, 0.3859, 0.4886, 0.4198,
        0.6582, 0.4538, 0.3808, 0.6788, 0.6328, 0.8631, 0.4983, 0.5490, 0.3519,
        0.5631, 0.4594, 0.5365, 0.5248, 0.4451, 0.4558, 0.4793, 0.6389, 0.7234,
        0.3734, 0.3457, 0.5360, 0.4281, 1.1684, 0.5294, 0.4066, 0.3297, 0.3671,
        0.6113, 0.4425, 0.4684, 0.6815, 0.3859, 0.6053, 0.3822, 0.3913, 1.2453,
        0.4409, 0.3566, 0.4454, 0.4176, 0.4760, 0.4971, 0.5012, 0.4316, 0.3986,
        0.3751, 0.3391, 0.3834, 0.4656, 0.4162, 0.7432, 0.5051, 0.6335, 0.4490,
        0.3647, 0.4829, 0.8224, 0.3247, 0.3659, 0.4589, 0.3837, 0.4440, 0.4366,
        0.5135, 0.5253, 0.4228, 0.3761, 0.4436, 0.4647, 0.5121, 0.4203, 0.4344,
        0.3311, 0.6444, 0.4212, 0.5446, 0.2708, 0.4052, 0.3880, 0.3720, 0.7506,
        0.4735, 0.4232, 0.3548, 0.5058, 

In [17]:
for dst, src in zip(dst_layers, src_layers):
    if (dst_state[dst] != src_state[src]).sum() > 0:
        print(dst, src)

In [18]:
torch.save(dst_state, '/home/allen/CenterNet_CornerNet-Squeeze_COCO.pth')